In [1]:
import numpy as nmp
from scipy.optimize import OptimizeResult, linprog
from dataclasses import dataclass

In [4]:
@dataclass
class Group:
    lb: nmp.ndarray
    ub: nmp.ndarray
    fun: float
    x: nmp.ndarray

    def bounds(self):
        return tuple(zip(self.lb, self.ub))

    def with_new_bounds(self, i, lb=None, ub=None):
        nlb = self.lb.copy()
        if lb is not None:
            nlb[i] = lb
        nub = self.ub.copy()
        if ub is not None:
            nub[i] = ub
        return Group(
            nlb,
            nub,
            self.fun,
            self.x
        )

    @classmethod
    def empty(cls, n, fun, x):
        return Group(
            nmp.zeros(n),
            nmp.array([None for i in range(n)]),
            fun,
            x
        )


def bounds(c, a, b):
    def calc_value(group: Group) -> tuple[float, nmp.ndarray]:
        res = linprog(
            c=c,
            A_ub=a,
            b_ub=b,
            method='simplex',
            bounds=group.bounds()
        )

        if res.success:
            return res.fun, res.x
        else:
            return float('inf'), nmp.array([])

    g_0: OptimizeResult = linprog(
        c=c,
        A_ub=a,
        b_ub=b,
        method='simplex'
    )
    g_0: Group = Group.empty(len(c), g_0.fun, g_0.x)
    i = 0
    while (True):
        i = nmp.argmax([not f.is_integer() for f in g_0.x])

        if i == -1:
            print('Can\'t find non-int x. Exiting.')
            return None

        ub = int(g_0.x[i])
        print(i)
        lb = ub + 1
        gn_1 = g_0.with_new_bounds(i, ub=ub)
        gn_1.fun, gn_1.x = calc_value(gn_1)
        print(gn_1)
        gn_2 = g_0.with_new_bounds(i, lb=lb)
        gn_2.fun, gn_2.x = calc_value(gn_2)
        print(gn_2)
        if gn_1.fun < gn_2.fun:
            if all([f.is_integer() for f in gn_1.x]):
                return gn_1.x

            g_0 = gn_1
        else:
            if all([f.is_integer() for f in gn_2.x]):
                return gn_2.x

            g_0 = gn_2
        i += 1


bounds(
    nmp.array([-1, -1]),
    nmp.array([
        [2, 11],
        [1, 1],
        [4, -5]
    ]),
    nmp.array([38, 7, 5])
)

0
Group(lb=array([0., 0.]), ub=array([4, None], dtype=object), fun=-6.727272727272727, x=array([4.        , 2.72727273]))
Group(lb=array([5., 0.]), ub=array([None, None], dtype=object), fun=inf, x=array([], dtype=float64))
1
Group(lb=array([0., 0.]), ub=array([4, 2], dtype=object), fun=-5.75, x=array([3.75, 2.  ]))
Group(lb=array([0., 3.]), ub=array([4, None], dtype=object), fun=-5.5, x=array([2.5, 3. ]))
0
Group(lb=array([0., 0.]), ub=array([3, 2], dtype=object), fun=-5.0, x=array([3., 2.]))
Group(lb=array([4., 0.]), ub=array([4, 2], dtype=object), fun=inf, x=array([], dtype=float64))


array([3., 2.])